In [36]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100)

In [2]:
def asset_growth(ticker):
    """Asset growth is defined as the YoY % change in quarterly total assets.
    Higher is worse, so multiply by -1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['Asset_growth'] = df.groupby('Quarter')['Total Assets'].pct_change()*-1
    return df[['Calendar_Year', 'Quarter','Total Assets','Asset_growth']]

In [3]:
asset_growth('AAPL').tail()

,Calendar_Year,Quarter,Total Assets,Asset_growth
4,2021,Q2,"337,158.00",-0.05
3,2021,Q3,"329,840.00",-0.04
2,2021,Q4,"351,002.00",-0.08
1,2022,Q1,"381,191.00",-0.08
0,2022,Q2,"350,662.00",-0.04


In [4]:
def asset_turnover_change(ticker):
    """YoY % change in the ratio of Sales to Total Assets. Higher is better"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    
    def divide_two_cols(df):
        df['Asset_Turnover'] = df['Revenue'] / (df['Total Assets'])
        return df
        
    df = divide_two_cols(df)
    df['Asset_Turnover_chg'] = df.groupby('Quarter')['Asset_Turnover'].pct_change()
    
    df['Asset_growth'] = df.groupby('Quarter')['Total Assets'].pct_change()*-1

    return df

In [5]:
df_AAPL = asset_turnover_change('AAPL')
df_AAPL.tail()

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,Asset_Turnover,Asset_Turnover_chg,Asset_growth
4,2021-03-27,2021,Q2,"2,030,688.81",1.57,0.79,5.35,1.41,"89,584.00","38,079.00",...,21.48,22.67,29.35,93.53,-1.26,1.41,AAPL,0.27,0.46,-0.05
3,2021-06-26,2021,Q3,"2,213,535.58",1.65,0.81,4.90,1.31,"81,434.00","35,255.00",...,25.45,27.18,34.44,116.50,-3.59,1.31,AAPL,0.25,0.31,-0.04
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,29.47,29.06,38.39,142.68,-6.43,1.25,AAPL,0.24,0.19,-0.08
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,21.34,23.85,41.09,66.93,0.31,2.11,AAPL,0.33,0.03,-0.08
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,28.63,29.44,42.49,111.65,-1.04,1.53,AAPL,0.28,0.04,-0.04


In [6]:
def cap_spend_growth(ticker):
    """YoY % change of: (Latest 4 quarters of cap ex + R&D expense)/(latest 4 quarters of sales)*-1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['TTM_expenses'] = (df['R&D Expenses'].rolling(4).sum()+df['CAPEX'].rolling(4).sum()*-1)
    df['TTM_sales'] = df['Revenue'].rolling(4).sum()
    df['cap_spend_%_of_rev'] = df['TTM_expenses']/df['TTM_sales']
    df['cap_spend_growth'] = df.groupby('Quarter')['cap_spend_%_of_rev'].pct_change()
    return df
    

In [7]:
cap_spend_growth('AAPL').tail()

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PS,PB,Price To FCF,PEG,EPS,Ticker,TTM_expenses,TTM_sales,cap_spend_%_of_rev,cap_spend_growth
4,2021-03-27,2021,Q2,"2,030,688.81",1.57,0.79,5.35,1.41,"89,584.00","38,079.00",...,22.67,29.35,93.53,-1.26,1.41,AAPL,"29,279.00","325,406.00",0.09,-0.08
3,2021-06-26,2021,Q3,"2,213,535.58",1.65,0.81,4.90,1.31,"81,434.00","35,255.00",...,27.18,34.44,116.50,-3.59,1.31,AAPL,"30,766.00","347,155.00",0.09,-0.07
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,29.06,38.39,142.68,-6.43,1.25,AAPL,"32,999.00","365,817.00",0.09,-0.05
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,23.85,41.09,66.93,0.31,2.11,AAPL,"33,445.00","378,323.00",0.09,-0.08
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,29.44,42.49,111.65,-1.04,1.53,AAPL,"34,815.00","386,017.00",0.09,0.00


In [8]:
def cap_spend_level(ticker):
    """(latest 4 quarters of cap ex + R&D expense)/latest 4 qtrs of sales*-1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['TTM_expenses'] = (df['R&D Expenses'].rolling(4).sum() +df['CAPEX'].rolling(4).sum()*-1)
    df['TTM_sales'] = df['Revenue'].rolling(4).sum()
    df['cap_spend_%_of_rev'] = df['TTM_expenses']/df['TTM_sales']
    return df[::-1]

In [9]:
cap_spend_level('AAPL').head(10)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,TTM_expenses,TTM_sales,cap_spend_%_of_rev
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,28.63,29.44,42.49,111.65,-1.04,1.53,AAPL,"34,815.00","386,017.00",0.09
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,21.34,23.85,41.09,66.93,0.31,2.11,AAPL,"33,445.00","378,323.00",0.09
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,29.47,29.06,38.39,142.68,-6.43,1.25,AAPL,"32,999.00","365,817.00",0.09
3,2021-06-26,2021,Q3,"2,213,535.58",1.65,0.81,4.90,1.31,"81,434.00","35,255.00",...,25.45,27.18,34.44,116.50,-3.59,1.31,AAPL,"30,766.00","347,155.00",0.09
4,2021-03-27,2021,Q2,"2,030,688.81",1.57,0.79,5.35,1.41,"89,584.00","38,079.00",...,21.48,22.67,29.35,93.53,-1.26,1.41,AAPL,"29,279.00","325,406.00",0.09
5,2020-12-26,2021,Q1,"2,314,861.46",1.50,0.81,6.58,1.70,"111,439.00","44,328.00",...,20.13,20.77,34.96,65.65,0.16,1.70,AAPL,"28,166.00","294,135.00",0.10
6,2020-09-26,2020,Q4,"1,915,229.78",1.51,0.80,3.79,0.74,"64,698.00","24,689.00",...,37.78,29.60,29.31,101.92,2.82,0.74,AAPL,"26,061.00","274,515.00",0.09
7,2020-06-27,2020,Q3,"1,525,055.21",1.30,0.77,3.46,0.65,"59,685.00","22,680.00",...,33.88,25.55,21.10,103.70,29.14,0.65,AAPL,"26,186.00","273,857.00",0.10
8,2020-03-28,2020,Q2,"1,080,171.45",1.14,0.76,3.34,0.64,"58,313.00","22,370.00",...,24.01,18.52,13.77,94.27,-0.49,0.65,AAPL,"26,120.00","267,981.00",0.10
9,2019-12-28,2020,Q1,"1,279,478.54",1.04,0.74,5.20,1.26,"91,819.00","35,217.00",...,14.39,13.93,14.29,45.04,0.22,1.26,AAPL,"26,013.00","267,683.00",0.10


In [10]:
def working_capital_accruals(ticker):
    """Trailing 4 quarters: 
    chg in receivables + chg in inventories + chg in accts payable + chg in tax accruals + change in misc. assets and liabilites
    Scaled by avg Total Assets
    """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['chgReceivables'] = df.groupby(['Quarter'])['Receivables'].diff().fillna(0)      
    df['chgInventory'] = df.groupby(['Quarter'])['Inventory'].diff().fillna(0)                      
    df['chgPayables'] = df.groupby(['Quarter'])['Payables'].diff().fillna(0)  
    df['chgTaxAcc'] = df.groupby(['Quarter'])['Tax Assets'].diff().fillna(0)
    #df['chgMisc'] = 
    df['Accruals'] = df['chgReceivables'] + df['chgInventory'] + df['chgPayables'] + df['chgTaxAcc']
    df['avgTotalAssets'] = df['Total Assets'].rolling(4).mean().fillna(0)
    df['avgAssetAccruals'] = (df['Accruals'] / df['avgTotalAssets']).fillna(0)
    return df
    

In [11]:
working_capital_accruals('AAPL').tail(3)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PEG,EPS,Ticker,chgReceivables,chgInventory,chgPayables,chgTaxAcc,Accruals,avgTotalAssets,avgAssetAccruals
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,-6.43,1.25,AAPL,"14,061.00","2,519.00","12,467.00",0.00,"29,047.00","343,013.50",0.08
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,0.31,2.11,AAPL,"6,633.00",903.00,"10,516.00",0.00,"18,052.00","349,797.75",0.05
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,-1.04,1.53,AAPL,"12,364.00",241.00,"12,555.00",0.00,"25,160.00","353,173.75",0.07


## Momentum

In [12]:
def RelativeStrengthIndex(ticker, time_window):
    """RSI compares the magnitude of recent price changes across stocks
    to identify stocks as overbought or oversold.
    A high RSI (usually above 70) indicate overbought and a low RSI 
    (typically below 30) indicates oversold.
    If first computes the avg price change for a given # (often 14)
    of peior trading days with rising prices and falling prices"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    diff = df['change'].dropna()
    
    # preserves dimensions of diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change  is equal to the positive diff, otherwise 0
    up_chg[diff > 0] = diff[diff >0]
    
    # down chg is equal to negative diff, oterhwise 0
    down_chg[diff < 0] = diff[diff < 0]
    
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods = time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods = time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi
    
    
    

In [13]:
RelativeStrengthIndex('AAPL',14).to_frame().tail()

,change
4,70.08
3,70.94
2,67.73
1,65.13
0,63.49


In [14]:
aapl = pd.read_csv('/Users/brendan/Desktop/Python/BoostedAI/prices/AAPL.csv')
aapl['RSI'] = RelativeStrengthIndex('AAPL',14)
aapl.head()


,Unnamed: 0,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,Ticker,RSI
0,0,2022-07-26,152.26,153.09,150.80,151.60,151.60,"52,295,393.00","52,295,393.00",-0.67,-0.44,151.83,"July 26, 22",-0.00,AAPL,63.49
1,1,2022-07-25,154.01,155.04,152.28,152.95,152.95,"53,518,694.00","53,518,694.00",-1.06,-0.69,153.42,"July 25, 22",-0.01,AAPL,65.13
2,2,2022-07-22,155.39,156.28,153.41,154.09,154.09,"66,626,929.00","66,626,929.00",-1.30,-0.84,154.59,"July 22, 22",-0.01,AAPL,67.73
3,3,2022-07-21,154.50,155.57,151.94,155.35,155.35,"64,957,826.00","64,957,826.00",0.85,0.55,154.29,"July 21, 22",0.01,AAPL,70.94
4,4,2022-07-20,151.12,153.72,150.37,153.04,153.04,"64,759,500.00","64,759,500.00",1.92,1.27,152.38,"July 20, 22",0.01,AAPL,70.08


In [15]:
def price_momentum(ticker):
    """This factor computes the total return for a given number of prior days.
        It is common to use the last 12 months except for the most recent month
    """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    df['Momentum'] = df['changePercent'].shift(1)/df['changePercent'].shift(12)-1
    return df
    

In [16]:
def price_momentum_vol_adj(ticker):
    """The indicator normalizes the total return over the previous 12 months by dividing it by the standard deviation of these returns."""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    df['MoVolAdj'] = (df['adjClose'].shift(0)/df['adjClose'].shift(252)-1)/df['changePercent'].rolling(252).std()
    return df

In [17]:
aapl = price_momentum_vol_adj('AAPL')
len(aapl.sort_values(by=['MoVolAdj']).dropna())

1006

## Value

In [37]:
def adj_book_to_price(ticker):
    """"The Residual (Actual - Estimate) after regressing Book-to-price (depend var) over ROE (independent var).
    Note: Total Equity is adjusted for Intangibles and Goodwill
    """
    df_price = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    #df = df[::-1]
    df_data = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df_merged = pd.concat([df_price,df_data], axis=1)
    return df_merged

In [40]:
adj_book_to_price('AVGO').head()

,Unnamed: 0,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,Ticker,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,0,2022-07-26,514.63,516.81,508.92,511.09,511.09,"1,781,649.00","1,781,649.00",-3.54,-0.69,512.27,"July 26, 22",-0.01,AVGO,2022-03-26,"2,022.00",Q2,"235,150.80",1.87,0.71,19.86,6.35,"8,103.00","5,439.00","1,261.00","2,045.00","3,394.00","2,590.00","9,005.00","3,083.00","1,668.00","14,810.00",0.00,"56,909.00","9,241.00","71,719.00","1,069.00",0.00,0.00,"6,910.00","39,164.00","43,819.00","50,729.00","20,963.00","4,243.00",-310.00,"-5,147.00",-85.00,"4,158.00","-1,750.00",0.67,0.42,6.55,0.32,0.01,2.14,90.59,36.11,54.48,0.04,0.12,0.05,22.70,29.02,11.22,56.55,2.49,6.35,AVGO
1,1,2022-07-25,510.46,513.32,506.53,513.11,513.11,"1,502,740.00","1,502,740.00",2.65,0.52,510.99,"July 25, 22",0.01,AVGO,2021-12-25,"2,022.00",Q1,"241,382.56",1.71,0.69,18.70,6.00,"7,706.00","5,049.00","1,206.00","1,924.00","3,108.00","2,472.00","10,219.00","2,539.00","1,520.00","15,341.00",0.00,"57,883.00","10,244.00","73,224.00","1,078.00","3,027.00",0.00,"6,287.00","39,205.00","43,943.00","50,230.00","22,994.00","3,486.00",-309.00,"-5,121.00",-101.00,"3,385.00","-1,764.00",0.66,0.40,7.64,0.32,0.01,2.44,81.14,36.51,44.63,0.03,0.11,0.05,24.41,31.32,10.50,71.31,0.97,5.82,AVGO
2,2,2022-07-22,518.81,519.79,506.68,512.52,512.52,"1,784,265.00","1,784,265.00",-6.29,-1.21,513.00,"July 22, 22",-0.01,AVGO,2021-09-25,"2,021.00",Q4,"217,556.60",1.58,0.67,17.98,4.83,"7,407.00","4,638.00","1,200.00","2,031.00","2,581.00","1,989.00","12,163.00","2,071.00","1,297.00","16,586.00",0.00,"58,984.00","11,374.00","75,570.00","1,086.00","2,619.00",566.00,"6,281.00","39,440.00","44,300.00","50,581.00","24,989.00","3,541.00",50.00,"-2,533.00",-88.00,"3,453.00","-1,561.00",0.63,0.35,5.95,0.27,0.01,2.64,67.32,35.30,32.02,0.03,0.08,0.04,27.34,29.37,8.71,63.01,4.44,4.65,AVGO
3,3,2022-07-21,512.55,517.81,506.02,517.68,517.68,"1,388,314.00","1,388,314.00",5.13,1.00,513.84,"July 21, 22",0.01,AVGO,2021-06-26,"2,021.00",Q3,"199,203.48",1.65,0.68,16.49,4.56,"6,778.00","4,197.00","1,205.00","2,045.00","2,126.00","1,876.00","11,105.00","2,234.00","1,160.00","15,636.00",0.00,"60,244.00","12,719.00","75,880.00",968.00,"3,008.00",620.00,"6,501.00","40,178.00","45,012.00","51,513.00","24,367.00","3,541.00",-47.00,"-1,907.00",-115.00,"3,426.00","-1,556.00",0.62,0.31,5.12,0.28,0.01,2.41,70.11,33.75,36.36,0.02,0.08,0.03,26.55,29.39,8.18,58.14,1.00,4.38,AVGO
4,4,2022-07-20,509.54,517.61,504.47,511.77,511.77,"2,740,636.00","2,740,636.00",2.23,0.44,511.28,"July 20, 22",0.00,AVGO,2021-03-27,"2,021.00",Q2,"184,107.27",1.68,0.68,16.16,3.65,"6,610.00","4,057.00","1,238.00","2,057.00","1,975.00","1,493.00","9,518.00","2,425.00","1,004.00","14,245.00",0.00,"61,279.00","14,068.00","75,524.00",830.00,"3,409.00",688.00,"6,436.00","40,160.00","45,122.00","51,558.00","23,966.00","3,569.00",-126.00,"-3,477.00",-126.00,"3,443.00","-1,552.00",0.61,0.30,4.24,0.23,0.01,2.21,68.41,29.26,39.15,0.02,0.06,0.03,30.83,27.85,7.68,53.47,3.79,3.46,AVGO


In [41]:
def total_yield(ticker):
    """ 75%* Indicated Annual Dividend + 25%* YoY %chg in quarterly share count"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df['DivYieldAnnual'] = df['Dividend Yield']*4
    df['Share_Count'] = df['Net Income']*df['NI per Share']
    df['share_count_change'] = df.groupby('Quarter')['Share_Count'].pct_change()
    df['TotalYield'] = .75*df['DivYieldAnnual'] + .25*df['share_count_change']
    return df

In [42]:
total_yield('AVGO')

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,DivYieldAnnual,Share_Count,share_count_change,TotalYield
0,2022-03-26,2022,Q2,"235,150.80",1.87,0.71,19.86,6.35,"8,103.00","5,439.00","1,261.00","2,045.00","3,394.00","2,590.00","9,005.00","3,083.00","1,668.00","14,810.00",0.00,"56,909.00","9,241.00","71,719.00","1,069.00",0.00,0.00,"6,910.00","39,164.00","43,819.00","50,729.00","20,963.00","4,243.00",-310.00,"-5,147.00",-85.00,"4,158.00","-1,750.00",0.67,0.42,6.55,0.32,0.01,2.14,90.59,36.11,54.48,0.04,0.12,0.05,22.70,29.02,11.22,56.55,2.49,6.35,AVGO,0.03,"16,441.42",NaN,NaN
1,2021-12-25,2022,Q1,"241,382.56",1.71,0.69,18.70,6.00,"7,706.00","5,049.00","1,206.00","1,924.00","3,108.00","2,472.00","10,219.00","2,539.00","1,520.00","15,341.00",0.00,"57,883.00","10,244.00","73,224.00","1,078.00","3,027.00",0.00,"6,287.00","39,205.00","43,943.00","50,230.00","22,994.00","3,486.00",-309.00,"-5,121.00",-101.00,"3,385.00","-1,764.00",0.66,0.40,7.64,0.32,0.01,2.44,81.14,36.51,44.63,0.03,0.11,0.05,24.41,31.32,10.50,71.31,0.97,5.82,AVGO,0.03,"14,832.00",NaN,NaN
2,2021-09-25,2021,Q4,"217,556.60",1.58,0.67,17.98,4.83,"7,407.00","4,638.00","1,200.00","2,031.00","2,581.00","1,989.00","12,163.00","2,071.00","1,297.00","16,586.00",0.00,"58,984.00","11,374.00","75,570.00","1,086.00","2,619.00",566.00,"6,281.00","39,440.00","44,300.00","50,581.00","24,989.00","3,541.00",50.00,"-2,533.00",-88.00,"3,453.00","-1,561.00",0.63,0.35,5.95,0.27,0.01,2.64,67.32,35.30,32.02,0.03,0.08,0.04,27.34,29.37,8.71,63.01,4.44,4.65,AVGO,0.03,"9,602.24",NaN,NaN
3,2021-06-26,2021,Q3,"199,203.48",1.65,0.68,16.49,4.56,"6,778.00","4,197.00","1,205.00","2,045.00","2,126.00","1,876.00","11,105.00","2,234.00","1,160.00","15,636.00",0.00,"60,244.00","12,719.00","75,880.00",968.00,"3,008.00",620.00,"6,501.00","40,178.00","45,012.00","51,513.00","24,367.00","3,541.00",-47.00,"-1,907.00",-115.00,"3,426.00","-1,556.00",0.62,0.31,5.12,0.28,0.01,2.41,70.11,33.75,36.36,0.02,0.08,0.03,26.55,29.39,8.18,58.14,1.00,4.38,AVGO,0.03,"8,562.96",NaN,NaN
4,2021-03-27,2021,Q2,"184,107.27",1.68,0.68,16.16,3.65,"6,610.00","4,057.00","1,238.00","2,057.00","1,975.00","1,493.00","9,518.00","2,425.00","1,004.00","14,245.00",0.00,"61,279.00","14,068.00","75,524.00",830.00,"3,409.00",688.00,"6,436.00","40,160.00","45,122.00","51,558.00","23,966.00","3,569.00",-126.00,"-3,477.00",-126.00,"3,443.00","-1,552.00",0.61,0.30,4.24,0.23,0.01,2.21,68.41,29.26,39.15,0.02,0.06,0.03,30.83,27.85,7.68,53.47,3.79,3.46,AVGO,0.03,"5,450.00",-0.67,-0.14
5,2020-12-26,2021,Q1,"189,963.18",1.71,0.69,16.35,3.39,"6,655.00","3,952.00","1,211.00","2,044.00","1,837.00","1,378.00","9,552.00","2,524.00",952.00,"14,300.00",0.00,"62,672.00","15,419.00","76,972.00",898.00,"3,098.00",735.00,"6,694.00","41,068.00","46,279.00","52,973.00","23,999.00","3,113.00",-122.00,"-1,057.00",-114.00,"2,999.00","-1,543.00",0.59,0.28,3.22,0.21,0.01,2.14,65.83,29.90,35.93,0.02,0.06,0.03,34.46,28.54,7.92,63.34,9.68,3.20,AVGO,0.03,"4,665.56",-0.69,-0.15
6,2020-09-26,2020,Q4,"141,600.15",1.68,0.69,15.97,3.27,"6,467.00","3,747.00","1,182.00","2,186.00","1,526.00","1,324.00","7,618.00","2,297.00","1,003.00","11,895.00",0.00,"64,038.00","16,782.00","75,933.00",836.00,"2,620.00",823.00,"6,371.00","40,235.00","45,661.00","52,032.00","23,901.00","3,348.00",-100.00,"-4,487.00",-102.00,"3,246.00","-1,395.00",0.58,0.24,3.63,0.20,0.01,1.87,65.15,27.66,37.49,0.02,0.06,0.02